## IPL First Innings Score Prediction

In this Project ,  We mainly focus on all regression algorithms 

Aim of this project is to predict the First Inning Score based on various features

#### Import Libraries

In [32]:
!pip install catboost

     |████████████████████████████████| 65.8MB 63kB/s 


In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stat
%matplotlib inline

### Feature Processing
from sklearn.preprocessing import StandardScaler
from datetime import datetime

### Model Building
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import  XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

### Metrics
import sklearn.metrics as metrics

### Hyper Paramter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

### Loading and Saving Model
import pickle

### Ensemble Technique - Combining Multiple models
## We call the algorithms like this way
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

## Then we use Voting Classifier for ensemble learning:
from sklearn.ensemble import VotingRegressor

In [2]:
# Loading the dataset
df = pd.read_csv('ipl.csv')
df.head()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
0,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,0,0.1,1,0,0,0,222
1,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,0,0.2,1,0,0,0,222
2,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.2,2,0,0,0,222
3,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.3,2,0,0,0,222
4,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.4,2,0,0,0,222


In [3]:
df.shape

(76014, 15)

### Data Cleaning --- Removing unwanted columns

In [4]:
columns_to_remove = ['mid', 'venue', 'batsman', 'bowler', 'striker', 'non-striker']
df.drop(labels=columns_to_remove, axis=1, inplace=True)

In [5]:
df.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total
0,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.1,1,0,222
1,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.2,1,0,222
2,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.2,2,0,222
3,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.3,2,0,222
4,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.4,2,0,222


In [6]:
df['bat_team'].unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
       'Mumbai Indians', 'Deccan Chargers', 'Kings XI Punjab',
       'Royal Challengers Bangalore', 'Delhi Daredevils',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant'], dtype=object)

In [7]:
consistent_teams = ['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
                    'Mumbai Indians', 'Kings XI Punjab', 'Royal Challengers Bangalore',
                    'Delhi Daredevils', 'Sunrisers Hyderabad']

In [8]:
df = df[(df['bat_team'].isin(consistent_teams)) & (df['bowl_team'].isin(consistent_teams))]

In [9]:
# Removing the first 5 overs data in every match
df = df[df['overs']>=5.0]

In [10]:
df.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total
32,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,0,5.1,59,0,222
33,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.2,59,1,222
34,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.3,59,1,222
35,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.4,59,1,222
36,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.5,58,1,222


In [11]:
print(df['bat_team'].unique())
print(df['bowl_team'].unique())

['Kolkata Knight Riders' 'Chennai Super Kings' 'Rajasthan Royals'
 'Mumbai Indians' 'Kings XI Punjab' 'Royal Challengers Bangalore'
 'Delhi Daredevils' 'Sunrisers Hyderabad']
['Royal Challengers Bangalore' 'Kings XI Punjab' 'Delhi Daredevils'
 'Rajasthan Royals' 'Mumbai Indians' 'Chennai Super Kings'
 'Kolkata Knight Riders' 'Sunrisers Hyderabad']


In [12]:
df.shape

(40108, 9)

In [13]:
df.dtypes

date               object
bat_team           object
bowl_team          object
runs                int64
wickets             int64
overs             float64
runs_last_5         int64
wickets_last_5      int64
total               int64
dtype: object

In [18]:
# Converting the column 'date' from string into datetime object
df['date'] = df['date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
df.dtypes

date              datetime64[ns]
bat_team                  object
bowl_team                 object
runs                       int64
wickets                    int64
overs                    float64
runs_last_5                int64
wickets_last_5             int64
total                      int64
dtype: object

### Data Preprocessing 

In [19]:
# Converting categorical features using OneHotEncoding method
encoded_df = pd.get_dummies(data=df, columns=['bat_team', 'bowl_team'])
encoded_df.head()

,date,runs,wickets,overs,runs_last_5,wickets_last_5,total,bat_team_Chennai Super Kings,bat_team_Delhi Daredevils,bat_team_Kings XI Punjab,bat_team_Kolkata Knight Riders,bat_team_Mumbai Indians,bat_team_Rajasthan Royals,bat_team_Royal Challengers Bangalore,bat_team_Sunrisers Hyderabad,bowl_team_Chennai Super Kings,bowl_team_Delhi Daredevils,bowl_team_Kings XI Punjab,bowl_team_Kolkata Knight Riders,bowl_team_Mumbai Indians,bowl_team_Rajasthan Royals,bowl_team_Royal Challengers Bangalore,bowl_team_Sunrisers Hyderabad
32,2008-04-18,61,0,5.1,59,0,222,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
33,2008-04-18,61,1,5.2,59,1,222,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
34,2008-04-18,61,1,5.3,59,1,222,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
35,2008-04-18,61,1,5.4,59,1,222,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
36,2008-04-18,61,1,5.5,58,1,222,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [20]:
encoded_df.shape

(40108, 23)

In [21]:
encoded_df.columns

Index(['date', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5',
       'total', 'bat_team_Chennai Super Kings', 'bat_team_Delhi Daredevils',
       'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders',
       'bat_team_Mumbai Indians', 'bat_team_Rajasthan Royals',
       'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad',
       'bowl_team_Chennai Super Kings', 'bowl_team_Delhi Daredevils',
       'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders',
       'bowl_team_Mumbai Indians', 'bowl_team_Rajasthan Royals',
       'bowl_team_Royal Challengers Bangalore',
       'bowl_team_Sunrisers Hyderabad'],
      dtype='object')

### Rearranging columns

In [22]:
encoded_df = encoded_df[['date', 'bat_team_Chennai Super Kings', 'bat_team_Delhi Daredevils', 'bat_team_Kings XI Punjab',
              'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Rajasthan Royals',
              'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad',
              'bowl_team_Chennai Super Kings', 'bowl_team_Delhi Daredevils', 'bowl_team_Kings XI Punjab',
              'bowl_team_Kolkata Knight Riders', 'bowl_team_Mumbai Indians', 'bowl_team_Rajasthan Royals',
              'bowl_team_Royal Challengers Bangalore', 'bowl_team_Sunrisers Hyderabad',
              'overs', 'runs', 'wickets', 'runs_last_5', 'wickets_last_5', 'total']]

In [23]:
encoded_df.head()

,date,bat_team_Chennai Super Kings,bat_team_Delhi Daredevils,bat_team_Kings XI Punjab,bat_team_Kolkata Knight Riders,bat_team_Mumbai Indians,bat_team_Rajasthan Royals,bat_team_Royal Challengers Bangalore,bat_team_Sunrisers Hyderabad,bowl_team_Chennai Super Kings,bowl_team_Delhi Daredevils,bowl_team_Kings XI Punjab,bowl_team_Kolkata Knight Riders,bowl_team_Mumbai Indians,bowl_team_Rajasthan Royals,bowl_team_Royal Challengers Bangalore,bowl_team_Sunrisers Hyderabad,overs,runs,wickets,runs_last_5,wickets_last_5,total
32,2008-04-18,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5.1,61,0,59,0,222
33,2008-04-18,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5.2,61,1,59,1,222
34,2008-04-18,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5.3,61,1,59,1,222
35,2008-04-18,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5.4,61,1,59,1,222
36,2008-04-18,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5.5,61,1,58,1,222


### Train Test Split - Time Series Data

All data till 2016 is in Training , Data after 2016 is in Testing

In [24]:
X_train = encoded_df.drop(labels='total', axis=1)[encoded_df['date'].dt.year <= 2016]
X_test = encoded_df.drop(labels='total', axis=1)[encoded_df['date'].dt.year >= 2017]

y_train = encoded_df[encoded_df['date'].dt.year <= 2016]['total'].values
y_test = encoded_df[encoded_df['date'].dt.year >= 2017]['total'].values

In [25]:
# Removing the 'date' column
X_train.drop(labels='date', axis=True, inplace=True)
X_test.drop(labels='date', axis=True, inplace=True)

In [26]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(37330, 21) (37330,) (2778, 21) (2778,)


### Model Building - Without applying any Feature Transformation to Features

In [37]:
#creating a dictionary of the models(estimators) 
estimators = {
    'Linear Regression': [LinearRegression()],
    'Lasso' :[Lasso()],
    'Ridge' :[Ridge()],
    'KNN' :[KNeighborsRegressor()],
    'Decision Tree' :[DecisionTreeRegressor()],
    'Support Vector Regression':[SVR()],
    'Random Forest' :[RandomForestRegressor()],
    'Gradient Boosting':[GradientBoostingRegressor()],
    'XG Boost': [XGBRegressor(objective='reg:squarederror')],
    'LGBM':[LGBMRegressor()],
    'Cat Boost':[CatBoostRegressor(verbose=0, n_estimators=100)],
}

#writing a function to fit models above to the train dataset
def mfit(estimators, X_train, y_train):
    for m in estimators:
        estimators[m][0].fit(X_train, y_train)
        print(m+' fitted')

mfit(estimators, X_train, y_train)

Linear Regression fitted
Lasso fitted
Ridge fitted
KNN fitted
Decision Tree fitted
Support Vector Regression fitted
Random Forest fitted
Gradient Boosting fitted
XG Boost fitted
LGBM fitted
Cat Boost fitted


In [38]:
#applying the fitted models to the test dataset
def mpredict(estimators, X_test, y_test):
    outcome = dict()
    for m in estimators:
        y_pred = estimators[m][0].predict(X_test)
        outcome[m] = [round(metrics.r2_score(y_test, y_pred), 2), 
                      metrics.mean_absolute_error(y_test, y_pred),
                     
                     metrics.mean_squared_error(y_test, y_pred),
                     np.sqrt(metrics.mean_squared_error(y_test, y_pred))]
    return outcome

outcome = mpredict(estimators, X_test, y_test)

In [42]:
outcome

{'Cat Boost': [0.73, 13.007552470931664, 277.8481552807143, 16.66877785804089],
 'Decision Tree': [0.47,
  17.146508279337652,
  538.3041756659467,
  23.201383055023825],
 'Gradient Boosting': [0.73,
  12.867882893149789,
  277.95328337791193,
  16.671931003273492],
 'KNN': [0.6, 15.136357091432686, 400.4675017998561, 20.011684132022875],
 'LGBM': [0.73, 12.945875222091612, 277.6182922085903, 16.661881412631356],
 'Lasso': [0.74, 12.214053814850248, 262.3797366400716, 16.198139912967527],
 'Linear Regression': [0.75,
  12.118617546193294,
  251.00792310417438,
  15.843229566732106],
 'Random Forest': [0.66,
  13.86613102951764,
  340.20636936437086,
  18.444684040784512],
 'Ridge': [0.75, 12.11858440825125, 251.00850756690366, 15.843248011910426],
 'Support Vector Regression': [0.64,
  14.474241481155584,
  361.263827690966,
  19.006941565937588],
 'XG Boost': [0.72, 13.05965037720078, 288.16571383075734, 16.975444436914085]}

In [39]:
#printing the regression errors as metrics for the model evaluation
for m in outcome:
    print('------------------------'+m+'------------------------')
    print('R2 score', round(outcome[m][0],2))
    print('MAE', round(outcome[m][1],2))
    print('MSE', round(outcome[m][2],2))
    print('RMSE', round(outcome[m][3],2))

------------------------Linear Regression------------------------
R2 score 0.75
MAE 12.12
MSE 251.01
RMSE 15.84
------------------------Lasso------------------------
R2 score 0.74
MAE 12.21
MSE 262.38
RMSE 16.2
------------------------Ridge------------------------
R2 score 0.75
MAE 12.12
MSE 251.01
RMSE 15.84
------------------------KNN------------------------
R2 score 0.6
MAE 15.14
MSE 400.47
RMSE 20.01
------------------------Decision Tree------------------------
R2 score 0.47
MAE 17.15
MSE 538.3
RMSE 23.2
------------------------Support Vector Regression------------------------
R2 score 0.64
MAE 14.47
MSE 361.26
RMSE 19.01
------------------------Random Forest------------------------
R2 score 0.66
MAE 13.87
MSE 340.21
RMSE 18.44
------------------------Gradient Boosting------------------------
R2 score 0.73
MAE 12.87
MSE 277.95
RMSE 16.67
------------------------XG Boost------------------------
R2 score 0.72
MAE 13.06
MSE 288.17
RMSE 16.98
------------------------LGBM---------------

### Saving All Above models

In [40]:
estimators

{'Cat Boost': [<catboost.core.CatBoostRegressor at 0x7f919b7879b0>],
 'Decision Tree': [DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best')],
 'Gradient Boosting': [GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                            init=None, learning_rate=0.1, loss='ls', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                       

In [41]:
for m in estimators:

  # open a file, where you ant to store the data
  file = open(m+'_model.pkl', 'wb')

  # dump information to that file
  pickle.dump(estimators[m][0], file)

#### Model Building -- On Scaled Data

In [43]:
sc = StandardScaler()
X_train_st = sc.fit_transform(X_train)
X_test_st = sc.transform(X_test)

mfit(estimators, X_train_st, y_train)
outcome = mpredict(estimators, X_test_st, y_test)

### Outcome output
#printing the regression errors as metrics for the model evaluation
for m in outcome:
    print('------------------------'+m+'------------------------')
    print('R2 score', round(outcome[m][0],2))
    print('MAE', round(outcome[m][1],2))
    print('MSE', round(outcome[m][2],2))
    print('RMSE', round(outcome[m][3],2))

Linear Regression fitted
Lasso fitted
Ridge fitted
KNN fitted
Decision Tree fitted
Support Vector Regression fitted
Random Forest fitted
Gradient Boosting fitted
XG Boost fitted
LGBM fitted
Cat Boost fitted
------------------------Linear Regression------------------------
R2 score 0.75
MAE 12.09
MSE 250.24
RMSE 15.82
------------------------Lasso------------------------
R2 score 0.7
MAE 13.41
MSE 304.15
RMSE 17.44
------------------------Ridge------------------------
R2 score 0.75
MAE 12.12
MSE 251.02
RMSE 15.84
------------------------KNN------------------------
R2 score 0.18
MAE 23.3
MSE 833.56
RMSE 28.87
------------------------Decision Tree------------------------
R2 score 0.46
MAE 17.17
MSE 544.35
RMSE 23.33
------------------------Support Vector Regression------------------------
R2 score 0.72
MAE 13.66
MSE 287.74
RMSE 16.96
------------------------Random Forest------------------------
R2 score 0.67
MAE 13.83
MSE 334.13
RMSE 18.28
------------------------Gradient Boosting--------

### Hyper-parameter Optimization

Ridge Regressor Optimization

In [44]:
ridge=Ridge()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=5.61566e-20): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.41749e-18): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.45442e-18): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.40095e-18): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.62242e-18): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAl

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [45]:
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 40}
-328.4152792487924


In [46]:
prediction=ridge_regressor.predict(X_test)
print('R2 Score:',round(metrics.r2_score(y_test, prediction), 2))
print('MAE:', metrics.mean_absolute_error(y_test, prediction))
print('MSE:', metrics.mean_squared_error(y_test, prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, prediction)))

R2 Score: 0.75
MAE: 12.117294527005022
MSE: 251.03172964112716
RMSE: 15.843980864704651


Similarly we can do Hyper-parameter optimization of other Alogirithms 

Soon I will be adding code of Hyper parameter optimization for all above algorithms 

#### Ensemble Techique : Combining Multiple Models

Training on Without Scaled Data

In [48]:
lr = LinearRegression()
ridge = Ridge()
rf=RandomForestRegressor()
xg=XGBRegressor()
dt=DecisionTreeRegressor()
svr=SVR()
gbm=GradientBoostingRegressor()
lgbm=LGBMRegressor()
catboost=CatBoostRegressor(verbose=0, n_estimators=100)

Ens = VotingRegressor( 
    estimators= [('Linear',lrb),('Ridge',ridge),('RandomForestRegressor',rf),
                 ('XGBRegressor',xg),('DT',dt),('SVR',svr),
                 ('GBM',gbm),('Lgbm',lgbm),('Catboost',catboost)   
                ])

## Training the Ensemble learning:
Ens= Ens.fit(X_train , y_train)

pred = Ens.predict(X_test)
print("R2 score =", round(metrics.r2_score(y_test, pred), 2))
print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))

#Accuracy of Ensemble learning:
print('Accuracy score of Ensemble Learning is = {:.2f}'.format(Ens.score(X_test, y_test)),'%')

[15:47:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R2 score = 0.74
MAE: 12.36605336973758
MSE: 260.6798617204975
RMSE: 16.145583350269433
Accuracy score of Ensemble Learning is = 0.74 %


In [49]:
file = open('Combined_model_without_scaling.pkl', 'wb')

# dump information to that file
pickle.dump(Ens, file)

Training on Scaled Data

In [51]:
lr = LinearRegression()
ridge = Ridge()
rf=RandomForestRegressor()
xg=XGBRegressor()
dt=DecisionTreeRegressor()
svr=SVR()
gbm=GradientBoostingRegressor()
lgbm=LGBMRegressor()
catboost=CatBoostRegressor(verbose=0, n_estimators=100)

Ens = VotingRegressor( 
    estimators= [('Linear',lr),('Ridge',ridge),('RandomForestRegressor',rf),
                 ('XGBRegressor',xg),('DT',dt),('SVR',svr),
                 ('GBM',gbm),('Lgbm',lgbm),('Catboost',catboost)   
                ])

## Training the Ensemble learning:
Ens= Ens.fit(X_train_st , y_train)

pred = Ens.predict(X_test_st)
print("R2 score =", round(metrics.r2_score(y_test, pred), 2))
print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))

#Accuracy of Ensemble learning:
print('Accuracy score of Ensemble Learning is = {:.2f}'.format(Ens.score(X_test_st, y_test)),'%')

[15:50:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R2 score = 0.75
MAE: 12.258366468459824
MSE: 257.1545287057534
RMSE: 16.03603843552869
Accuracy score of Ensemble Learning is = 0.75 %


In [52]:
file = open('Combined_model_with_scaling.pkl', 'wb')

# dump information to that file
pickle.dump(Ens, file)